In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-content-based-filtering-recommender/data/content_item_train_header.txt
/kaggle/input/movie-content-based-filtering-recommender/data/content_item_vecs.csv
/kaggle/input/movie-content-based-filtering-recommender/data/content_movie_list.csv
/kaggle/input/movie-content-based-filtering-recommender/data/content_y_train.csv
/kaggle/input/movie-content-based-filtering-recommender/data/content_user_train_header.txt
/kaggle/input/movie-content-based-filtering-recommender/data/content_top10_df.csv
/kaggle/input/movie-content-based-filtering-recommender/data/content_user_to_genre.pickle
/kaggle/input/movie-content-based-filtering-recommender/data/content_item_train.csv
/kaggle/input/movie-content-based-filtering-recommender/data/content_bygenre_df.csv
/kaggle/input/movie-content-based-filtering-recommender/data/content_user_train.csv


In [2]:
from numpy import genfromtxt
import csv
from collections import defaultdict
import pickle
import tabulate
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [3]:
RANDOM_STATE = 1

In [4]:
top10_df = pd.read_csv("/kaggle/input/movie-content-based-filtering-recommender/data/content_top10_df.csv")
top10_df

,movie id,num ratings,ave rating,title,genres
0,4993,198,4.106061,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
1,5952,188,4.021277,"Lord of the Rings: The Two Towers, The",Adventure|Fantasy
2,7153,185,4.118919,"Lord of the Rings: The Return of the King, The",Action|Adventure|Drama|Fantasy
3,4306,170,3.867647,Shrek,Adventure|Animation|Children|Comedy|Fantasy|Ro...
4,58559,149,4.238255,"Dark Knight, The",Action|Crime|Drama
5,6539,149,3.778523,Pirates of the Caribbean: The Curse of the Bla...,Action|Adventure|Comedy|Fantasy
6,79132,143,4.066434,Inception,Action|Crime|Drama|Mystery|Sci-Fi|Thriller
7,6377,141,3.960993,Finding Nemo,Adventure|Animation|Children|Comedy
8,4886,132,3.871212,"Monsters, Inc.",Adventure|Animation|Children|Comedy|Fantasy
9,7361,131,4.160305,Eternal Sunshine of the Spotless Mind,Drama|Romance|Sci-Fi


In [5]:
bygenre_df = pd.read_csv("/kaggle/input/movie-content-based-filtering-recommender/data/content_bygenre_df.csv")
bygenre_df

,genre,num movies,ave rating/genre,ratings per genre
0,Action,321,3.37,10377
1,Adventure,234,3.42,8785
2,Animation,76,3.63,2588
3,Children,69,3.44,2472
4,Comedy,326,3.36,8911
5,Crime,139,3.54,4671
6,Documentary,13,3.81,280
7,Drama,342,3.61,10201
8,Fantasy,124,3.37,4468
9,Horror,56,3.20,1345


In [6]:
def load_data():
    item_train = genfromtxt("/kaggle/input/movie-content-based-filtering-recommender/data/content_item_train.csv", delimiter=",")
    user_train = genfromtxt("/kaggle/input/movie-content-based-filtering-recommender/data/content_user_train.csv", delimiter=",")
    y_train = genfromtxt("/kaggle/input/movie-content-based-filtering-recommender/data/content_y_train.csv", delimiter=",")
    with open("/kaggle/input/movie-content-based-filtering-recommender/data/content_item_train_header.txt", newline="") as f:
        item_features = list(csv.reader(f))[0]
        
    with open("/kaggle/input/movie-content-based-filtering-recommender/data/content_user_train_header.txt", newline="") as f:
        user_features = list(csv.reader(f))[0]
    
    item_vecs = genfromtxt("/kaggle/input/movie-content-based-filtering-recommender/data/content_item_vecs.csv", delimiter = ",")
    movie_dict = defaultdict(dict)
    count = 0 
    with open("/kaggle/input/movie-content-based-filtering-recommender/data/content_movie_list.csv", newline="") as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for line in reader:
            if count == 0:
                count+=1
            else:
                count+=1
                movie_id = int(line[0])
                movie_dict[movie_id]["title"] = line[1]
                movie_dict[movie_id]["genres"] = line[2]
    
    with open("/kaggle/input/movie-content-based-filtering-recommender/data/content_user_to_genre.pickle", 'rb') as f:
        user_to_genre = pickle.load(f)
    
    return item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre
    
# file = "/kaggle/input/movie-content-based-filtering-recommender/data/content_item_train_header.txt"
# with open(file, 'r') as file:
#     content = file.read()
# content

In [7]:
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3   # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1   # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items

print("No of training vectors: ",len(item_train))
item_train[:2]

No of training vectors:  50884


array([[6.87400000e+03, 2.00300000e+03, 3.96183206e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00],
       [8.79800000e+03, 2.00400000e+03, 3.76136364e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00]])

In [8]:
def print_train(X_train, features, vs, u_s, maxcount=5, user=True):
    if user:
        flist = [".0f", ".0f", ".1f",
                 ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f"]
    else:
        flist = [".0f", ".0f", ".1f", 
                 ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f"]

    head = features[:vs]
    if vs<u_s:
        print(f"Error, vector start {vs} should be greater than user start {u_s}")

    for i in range(u_s):
        head[i] = "[" + head[i] + "]"
        
    genres = features[vs:]
    hdr = head + genres
    disp = [split_str(hdr, 5)]
    count = 0
    
    for i in range(0, X_train.shape[0]):
        if count == maxcount: break
        count += 1
        disp.append([X_train[i, 0].astype(int),
                     X_train[i, 1].astype(int),
                     X_train[i, 2].astype(float),
                     *X_train[i, 3:].astype(float)
                    ])
    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow", floatfmt=flist, numalign='center')
    return table

In [9]:
def split_str(ifeatures, smax):
    ''' split the feature name strings to tables fit '''
    ofeatures = []
    for s in ifeatures:
        if not ' ' in s:  # skip string that already have a space
            if len(s) > smax:
                mid = int(len(s)/2)
                s = s[:mid] + " " + s[mid:]
        ofeatures.append(s)
    return ofeatures
    

In [10]:
print_train(user_train, user_features, uvs, u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9


In [11]:
print_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)

[movie id],year,ave rating,Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
6874,2003,4.0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
8798,2004,3.8,1,0,0,0,0,1,0,1,0,0,0,0,0,1
46970,2006,3.2,1,0,0,0,1,0,0,0,0,0,0,0,0,0
48516,2006,4.3,0,0,0,0,0,1,0,1,0,0,0,0,0,1
58559,2008,4.2,1,0,0,0,0,1,0,1,0,0,0,0,0,0


In [12]:
y_train.shape

(50884,)

In [13]:
user_train

array([[  2.  ,  22.  ,   4.  , ...,   0.  ,   3.88,   3.89],
       [  2.  ,  22.  ,   4.  , ...,   0.  ,   3.88,   3.89],
       [  2.  ,  22.  ,   4.  , ...,   0.  ,   3.88,   3.89],
       ...,
       [610.  , 514.  ,   3.69, ...,   3.67,   3.61,   3.6 ],
       [610.  , 514.  ,   3.69, ...,   3.67,   3.61,   3.6 ],
       [610.  , 514.  ,   3.69, ...,   3.67,   3.61,   3.6 ]])

In [14]:
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1,1))
scalerTarget.fit(y_train.reshape(-1,1))
y_train = scalerTarget.transform(y_train.reshape(-1,1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


In [15]:
print_train(user_train, user_features, uvs, u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
-1,-1,0.9,0.8,1.2,-3.8,-3.5,0.8,0.7,0.9,0.7,-5.0,0.1,0.5,-4.9,0.6,0.6
-1,-1,0.9,0.8,1.2,-3.8,-3.5,0.8,0.7,0.9,0.7,-5.0,0.1,0.5,-4.9,0.6,0.6
-1,-1,0.9,0.8,1.2,-3.8,-3.5,0.8,0.7,0.9,0.7,-5.0,0.1,0.5,-4.9,0.6,0.6
-1,-1,0.9,0.8,1.2,-3.8,-3.5,0.8,0.7,0.9,0.7,-5.0,0.1,0.5,-4.9,0.6,0.6
-1,-1,0.9,0.8,1.2,-3.8,-3.5,0.8,0.7,0.9,0.7,-5.0,0.1,0.5,-4.9,0.6,0.6


In [16]:
item_train, item_test = train_test_split(item_train, train_size=0.8, shuffle=True, random_state=RANDOM_STATE)
user_train, user_test = train_test_split(user_train, train_size=0.8, shuffle=True, random_state=RANDOM_STATE)
y_train, y_test       = train_test_split(y_train, train_size=0.8, shuffle=True, random_state=RANDOM_STATE)

In [17]:
print("User Training:", user_train.shape)
print("User Test:", user_test.shape)

print("Item Training:", item_train.shape)
print("Item Test:", item_test.shape)

User Training: (40707, 17)
User Test: (10177, 17)
Item Training: (40707, 17)
Item Test: (10177, 17)


In [18]:
num_outputs = 32
tf.random.set_seed(RANDOM_STATE)
user_NN = Sequential([
    Dense(units = 256, activation = 'relu'),
    Dense(units = 128, activation = 'relu'),
    Dense(units = 32, activation = 'linear'),
])

In [19]:
user_NN.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)